# 数据收集

本报告的数据来源于3个不同的维度
* 项目提供的twitter_archive_enhanced.csv，其中包含了待分析的基本数据
* 项目提供的tweet_json.txt，其中包含了每条推特信息的转发数和点赞数等额外信息
* 项目同时提供了一个图像预测文件image-predictions.tsv的下载链接（https://raw.githubusercontent.com/udacity/new-dand-advanced-china/master/%E6%95%B0%E6%8D%AE%E6%B8%85%E6%B4%97/WeRateDogs%E9%A1%B9%E7%9B%AE/image-predictions.tsv）

其中twitter_archive_enhanced.csv和tweet_json.txt直接拷贝到了项目目录下
而image-predictions.tsv会使用RequestAPI的方式从网络中获取


In [27]:
# 先通过RequestAPI下载对应的文件，并且将原始文件存储到data_gathering目录中

import requests
import os
import pandas as pd
import json
import time

url = 'https://raw.githubusercontent.com/udacity/new-dand-advanced-china/master/%E6%95%B0%E6%8D%AE%E6%B8%85%E6%B4%97/WeRateDogs%E9%A1%B9%E7%9B%AE/image-predictions.tsv'
r = requests.get(url)

folder_name = 'data_gathering'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

with open(folder_name + '/image-predictions.tsv', 'wb') as f:
    f.write(r.content)

images = pd.read_csv('data_gathering/image-predictions.tsv', sep = '\t', encoding = 'utf-8')
images
images.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [28]:
# 读取对应的twitter_archive_enhanced.csv文件
archive = pd.read_csv('twitter-archive-enhanced.csv', encoding = 'utf-8')


In [29]:
#读取对应的tweet_json.txt文件，并且将数据按照需求，进行获取
#我们在twitter_archive文件中有的tweet才去获取它的扩展信息
with open('tweet_json.txt') as f:
    content = f.readlines()
    
json_list = [json.loads(line) for line in open('tweet_json.txt')]
json_tweet = [{'tweet_id': int(tweet_obj['id']),
        'date_time': pd.to_datetime(tweet_obj['created_at']),
        'favorites': tweet_obj['favorite_count'],
        'retweets': tweet_obj['retweet_count'],
        'user_followers': tweet_obj['user']['followers_count'],
        'user_favourites': tweet_obj['user']['favourites_count']} for tweet_obj in json_list]


json_tweet[9]

{'tweet_id': 890240255349198849,
 'date_time': Timestamp('2017-07-26 15:59:51'),
 'favorites': 32467,
 'retweets': 7684,
 'user_followers': 3768792,
 'user_favourites': 120162}

In [30]:
# 将处理过的字段的json文件转换成csv文件，便于分析
json_tweet = pd.DataFrame(json_tweet, columns = ['tweet_id', 'date_time', 'favorites', 'retweets',
                                               'user_followers', 'user_favourites'])

json_tweet.to_csv('new_tweet_json.txt', encoding = 'utf-8', index=False)
json_tweets_csv = pd.read_csv('new_tweet_json.txt', encoding = 'utf-8')


In [19]:
json_tweets_csv.user_followers.sort_values()

json_tweets_csv.favorites.sort_values()

json_tweets_csv.retweets.sort_values()

288         0
1291        2
271         3
339         3
112         3
29          4
1076        6
54          7
424        10
63         10
2294       14
2335       15
183        17
1519       19
176        20
209        23
2315       23
2185       25
1233       26
185        28
406        30
100        31
608        32
2220       34
2256       34
2255       37
963        37
282        37
2333       37
881        38
        ...  
935     23870
307     23870
526     23959
447     23959
886     24183
1073    24183
651     24370
1621    24370
152     24997
114     27502
621     27586
1762    30797
456     31140
300     31140
1826    31810
166     32589
130     32705
133     32705
866     33230
162     33231
534     40437
446     42045
443     42045
65      45655
410     47958
814     52101
1075    52101
531     56373
257     56373
1035    79116
Name: retweets, Length: 2352, dtype: int64

# 评估数据

在收集到数据之后，我们会通过编程和肉眼分析的方式，来初步定为数据的整洁度和数据质量问题

In [11]:
archive.info()
archive
archive['name'].value_counts()

archive['rating_denominator'].value_counts()
archive['rating_numerator'].value_counts()
archive['score_ratio'] = archive['rating_numerator']/archive['rating_denominator']

#archive.sample(5)

archive[archive['in_reply_to_user_id'].notnull()].sample(4)


archive['rating_numerator'].value_counts()
archive['rating_denominator'].value_counts()

archive.describe()


archive[archive['rating_numerator']==1776]

archive[archive.tweet_id.duplicated()]

archive.rating_numerator.sort_values()

archive

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,score_ratio
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None,1.3
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None,1.3
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None,1.2
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None,1.3
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None,1.2
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None,1.3
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None,1.3
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None,1.3
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None,1.3
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None,1.4


In [12]:
# 代码方式看看image数据集
images
images.info()
images['jpg_url'].value_counts()
images[images['jpg_url'] == 'https://pbs.twimg.com/media/CiibOMzUYAA9Mxz.jpg']

images[images['jpg_url'].duplicated()]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1297,752309394570878976,https://pbs.twimg.com/ext_tw_video_thumb/67535...,1,upright,0.303415,False,golden_retriever,0.181351,True,Brittany_spaniel,0.162084,True
1315,754874841593970688,https://pbs.twimg.com/media/CWza7kpWcAAdYLc.jpg,1,pug,0.272205,True,bull_mastiff,0.251530,True,bath_towel,0.116806,False
1333,757729163776290825,https://pbs.twimg.com/media/CWyD2HGUYAQ1Xa7.jpg,2,cash_machine,0.802333,False,schipperke,0.045519,True,German_shepherd,0.023353,True
1345,759159934323924993,https://pbs.twimg.com/media/CU1zsMSUAAAS0qW.jpg,1,Irish_terrier,0.254856,True,briard,0.227716,True,soft-coated_wheaten_terrier,0.223263,True
1349,759566828574212096,https://pbs.twimg.com/media/CkNjahBXAAQ2kWo.jpg,1,Labrador_retriever,0.967397,True,golden_retriever,0.016641,True,ice_bear,0.014858,False
1364,761371037149827077,https://pbs.twimg.com/tweet_video_thumb/CeBym7...,1,brown_bear,0.713293,False,Indian_elephant,0.172844,False,water_buffalo,0.038902,False
1368,761750502866649088,https://pbs.twimg.com/media/CYLDikFWEAAIy1y.jpg,1,golden_retriever,0.586937,True,Labrador_retriever,0.398260,True,kuvasz,0.005410,True
1387,766078092750233600,https://pbs.twimg.com/media/ChK1tdBWwAQ1flD.jpg,1,toy_poodle,0.420463,True,miniature_poodle,0.132640,True,Chesapeake_Bay_retriever,0.121523,True
1407,770093767776997377,https://pbs.twimg.com/media/CkjMx99UoAM2B1a.jpg,1,golden_retriever,0.843799,True,Labrador_retriever,0.052956,True,kelpie,0.035711,True
1417,771171053431250945,https://pbs.twimg.com/media/CVgdFjNWEAAxmbq.jpg,3,Samoyed,0.978833,True,Pomeranian,0.012763,True,Eskimo_dog,0.001853,True


In [13]:
json_tweets_csv

json_tweets_csv.info()

json_tweets_csv[json_tweets_csv['tweet_id'].duplicated()]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2352 entries, 0 to 2351
Data columns (total 6 columns):
tweet_id           2352 non-null int64
date_time          2352 non-null object
favorites          2352 non-null int64
retweets           2352 non-null int64
user_followers     2352 non-null int64
user_favourites    2352 non-null int64
dtypes: int64(5), object(1)
memory usage: 110.3+ KB


,tweet_id,date_time,favorites,retweets,user_followers,user_favourites


# 数据质量

数据质量从下面几个维度考虑：
* 完整性
* 有效性
* 准确性
* 一致性

### archive 表格

* 狗的名字存在一些错误的单词（such,quite,not,very,just,my,his,one,a,an）
* 需要过滤掉转发的记录，只保存原始评分的tweet记录
* 字段属性不正确（retweeted_status_id，retweeted_status_user_id）
* timestamp的数据类型应该为datetime，而不是object
* 对于“空字段”的表示，可以统一。（NaN，None）


### image 表格

* 被分析的图片的url有重复
* 数据缺失，archive中有2356条记录，而images中只有2075条记录


### tweet_json 表格

* 数据只有2352条，与archive中的数据不一致



# 数据整洁度

* 狗的种类，合并成一列，将具体的类型填写其中
* 应该将3个表格的信息进行合并
* 对于图片预测的概率（p1_conf,p2_conf,p3_conf），及预测的狗的种类(p1, p2, p3)进行合并

In [6]:
all_columns = pd.Series(list(archive) + list(images) + list(json_tweets_csv))

all_columns[all_columns.duplicated()]

17    tweet_id
29    tweet_id
dtype: object

In [15]:
list(archive)

['tweet_id',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'timestamp',
 'source',
 'text',
 'retweeted_status_id',
 'retweeted_status_user_id',
 'retweeted_status_timestamp',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'doggo',
 'floofer',
 'pupper',
 'puppo',
 'score_ratio']

In [16]:
list(images)

['tweet_id',
 'jpg_url',
 'img_num',
 'p1',
 'p1_conf',
 'p1_dog',
 'p2',
 'p2_conf',
 'p2_dog',
 'p3',
 'p3_conf',
 'p3_dog']

In [17]:
list(json_tweets_csv)

['tweet_id',
 'date_time',
 'favorites',
 'retweets',
 'user_followers',
 'user_favourites']

# 清理数据

根据上面分析的质量和整洁度问题，进行数据的清理

In [31]:
# 先将所有的数据列进行合并

df_all = pd.merge(archive, images, how = 'left', on = ['tweet_id'] )
df_all = pd.merge(df_all, json_tweets_csv, how = 'left', on = ['tweet_id'])

df_all.to_csv('df_all.csv', encoding = 'utf-8')
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 0 to 2355
Data columns (total 33 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
jpg_url                       20

In [32]:
# retweeted_status_id不为空，说明这条记录是被转发的记录
# 将转发的内容删除掉
df_all = df_all[pd.isnull(df_all.retweeted_status_id)]

# 对于没有提供图片分析数据的tweet记录删除掉
df_all = df_all.dropna(subset = ['jpg_url'])
len(df_all)

1994

In [33]:
# 将狗的状态进行合并
print(df_all.doggo.value_counts())
print(df_all.floofer.value_counts())
print(df_all.pupper.value_counts())
print(df_all.puppo.value_counts())

None     1920
doggo      74
Name: doggo, dtype: int64
None       1986
floofer       8
Name: floofer, dtype: int64
None      1782
pupper     212
Name: pupper, dtype: int64
None     1971
puppo      23
Name: puppo, dtype: int64


In [34]:
# 将狗的类型进行合并
melt_column = ['doggo', 'floofer', 'pupper', 'puppo']
stay_column = [x for x in df_all.columns.tolist() if x not in melt_column]

df_all = pd.melt(df_all, id_vars = stay_column, value_vars = melt_column, 
                         var_name = 'stages', value_name = 'dog_stage')


df_all = df_all.drop('stages', 1)
df_all = df_all.sort_values('dog_stage').drop_duplicates('tweet_id', keep = 'last')

print(len(df_all))

1994


In [35]:
# 计算狗狗的评分比率，得分/满分
df_all['score_ratio'] = df_all['rating_numerator']/df_all['rating_denominator']

In [36]:
df_all = df_all.drop(['retweeted_status_id','retweeted_status_user_id', 'retweeted_status_timestamp'], 1)

df_all = df_all.drop(['in_reply_to_status_id','in_reply_to_user_id'], 1)
df_all = df_all.drop(['p1','p1_conf','p2','p2_conf','p3','p3_conf','p2_dog','p3_dog','p1_dog'], 1)


df_all.to_csv('twitter_archive_master.csv', index=False, encoding = 'utf-8')

df_master = pd.read_csv('twitter_archive_master.csv')
df_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1994 entries, 0 to 1993
Data columns (total 17 columns):
tweet_id              1994 non-null int64
timestamp             1994 non-null object
source                1994 non-null object
text                  1994 non-null object
expanded_urls         1994 non-null object
rating_numerator      1994 non-null int64
rating_denominator    1994 non-null int64
name                  1994 non-null object
jpg_url               1994 non-null object
img_num               1994 non-null float64
date_time             1994 non-null object
favorites             1994 non-null float64
retweets              1994 non-null float64
user_followers        1994 non-null float64
user_favourites       1994 non-null float64
dog_stage             1994 non-null object
score_ratio           1994 non-null float64
dtypes: float64(6), int64(3), object(8)
memory usage: 264.9+ KB


In [37]:
# 因为想要分析转发和内容质量之间的关系，所以额外整理了一个转发数据相关的文件
# 并且按照每个月进行数据的统计

df_all['timestamp']=pd.to_datetime(df_all['timestamp'])
df_all['date_month']=pd.to_datetime(df_all['timestamp']).values.astype('datetime64[M]')

month_group = df_all.groupby('date_month')
plot_data_df = pd.DataFrame([], columns=['month','tweet_count', 'retweet_count', 'favourites_count'])

for name,group in month_group:
    nest_dict = pd.DataFrame([name,len(group.tweet_id), sum(group.retweets), sum(group.favorites)]).T
    nest_dict.columns = plot_data_df.columns
    plot_data_df = pd.concat([plot_data_df, nest_dict], ignore_index=True)
    
    
    
plot_data_df.to_csv('twitter_month_data.csv', index=False, encoding = 'utf-8')
df_master = pd.read_csv('twitter_month_data.csv')
df_master.info()    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 4 columns):
month               22 non-null object
tweet_count         22 non-null int64
retweet_count       22 non-null float64
favourites_count    22 non-null float64
dtypes: float64(2), int64(1), object(1)
memory usage: 784.0+ bytes
